In [1]:
# 使用bagging

In [3]:
import tensorflow as tf
import numpy as np
import sklearn.feature_selection as fs

TRAIN_DATA_FILE = "train_data.csv"
TEST_DATA_FILE = "test_data.csv"

training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=TRAIN_DATA_FILE,
    target_dtype=np.int,
    features_dtype=np.float32,
    target_column=0
)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=TEST_DATA_FILE,
    target_dtype=np.int,
    features_dtype=np.float32,
    target_column=0
)
# print(training_set)
X, y = training_set.data, training_set.target
print(X)
print(y)
X_test, y_test = test_set.data, test_set.target
print(X_test)
print(y_test)

[[  997.9039917    637.5949707    658.66802979 ..., -2203.02001953
  -1180.18994141   433.90600586]
 [  914.19799805   634.23999023   593.70501709 ..., -2250.         -1360.56005859
    524.07501221]
 [ 3800.81005859  1671.33996582  1206.88000488 ...,  2762.57006836
    150.93099976  3800.81005859]
 ..., 
 [ 5833.75976562  3465.73999023  1283.31994629 ...,  3759.70996094
   3267.16992188   388.3460083 ]
 [ 3079.31005859  2435.30004883  1444.40002441 ...,  1880.58996582
   1614.83996582  1518.97998047]
 [ 7352.56982422  6941.18994141  1667.86999512 ...,  5983.12988281
   1249.70996094  2424.22998047]]
[0 0 0 ..., 5 5 5]
[[ 2750.11010742  1361.89001465  -247.50799561 ...,   -81.54180145
   1146.26000977 -1566.48999023]
 [ 2646.25        1760.84997559   481.1000061  ...,   668.75        1519.73999023
   -568.97302246]
 [ 3509.62011719  3509.62011719  -241.32000732 ...,  1132.90002441
    594.059021    -993.65802002]
 ..., 
 [  987.46099854 -2364.60009766  -155.59199524 ..., -4259.29980469

In [84]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from collections import Counter

In [42]:
cls = BaggingClassifier(base_estimator=GaussianNB(),max_samples=1.0)

y_pred = cls.fit(X, y).predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred)
print(acc)


0.683333333333


In [58]:
# 打乱数据集

cls = BaggingClassifier(base_estimator=GaussianNB(), max_samples=1.0)

indices = np.random.permutation(X.shape[0])  # shape[0]表示第0轴的长度，通常是训练数据的数量  
rand_data_x = X[indices]
rand_data_y = y[indices]  # data_y就是标记（label）  

y_pred = cls.fit(rand_data_x, rand_data_y).predict(X_test)
print(type(y_pred))
acc = metrics.accuracy_score(y_test, y_pred)
print(acc)


<class 'numpy.ndarray'>
0.683333333333


In [43]:
# 我打算将数据手动分开再分别训练

In [75]:
# 提取出类别为1的数据
X_1 = np.array([list(X[i]) for i in range(len(y)) if y[i] == 1])
y_1 = np.array([y[i] for i in range(len(y)) if y[i] == 1])
print(X_1.shape[0], y_1.shape[0])
# 提取出类别不为1的数据
X_n1 = np.array([list(X[i]) for i in range(len(y)) if y[i] != 1])
y_n1 = np.array([y[i] for i in range(len(y)) if y[i] != 1])
print(len(X_n1), len(y_n1))

# 将类别为1的随机分成5份和其他类别数据再组成5份训练数据
X_5 = []  # 每个元素是一个X训练数据
y_5 = []  # 每个元素是一个y训练数据





7431 7431
3114 3114


In [87]:
pred_record = []
y_pred = None
al = int(len(X_1) / 5)
gnb = GaussianNB()
for i in range(5):
    xt1 = X_1[i * al:(i + 1) * al, :]
    yt1 = y_1[i * al:(i + 1) * al]
    X_temp = np.concatenate((xt1, X_n1), axis=0)
    y_temp = np.concatenate((yt1, y_n1), axis=0)
    # print(temp.shape)
    if i == 0:
        y_pred = gnb.fit(X_temp, y_temp).predict(X_test)
    else:
        y_p = gnb.fit(X_temp, y_temp).predict(X_test)
        y_pred = np.concatenate((y_pred, y_p))

        # acc = metrics.accuracy_score(y_test, y_pred)
        # print(acc)
y_pred = y_pred.reshape(5, -1)
print(y_pred.shape)

col_len = y_pred.shape[1]
# col_len = 5
y_pred_final = []
for i in range(col_len):
    choose = Counter(list(y_pred[:, i])).most_common(1)[0][0]
    # print(choose)
    y_pred_final.append(choose)

acc = metrics.accuracy_score(y_test, y_pred_final)
print(acc)


(5, 300)
0.71


In [74]:
al = int(len(X_1) / 4)
gnb = GaussianNB()
for i in range(4):
    xt1 = X_1[i * al:(i + 1) * al, :]
    yt1 = y_1[i * al:(i + 1) * al]
    X_temp = np.concatenate((xt1, X_n1), axis=0)
    y_temp = np.concatenate((yt1, y_n1), axis=0)
    # print(temp.shape)
    y_pred = gnb.fit(X_temp, y_temp).predict(X_test)
    
    # 记录每个结果,最后会评比
    
    acc = metrics.accuracy_score(y_test, y_pred)
    print(acc)


0.71
0.7
0.62
0.586666666667
